In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
# Load dataset diabetes
df = pd.read_csv('diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Pisahkan fitur dan target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [6]:
# Pisahkan data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Definisikan algoritma yang akan digunakan
algoritma = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('SVM kernel polynomial', SVC(kernel='poly', degree=2)),
    ('Decision Tree', DecisionTreeClassifier())
]

In [8]:
# Buat ensemble voting
ensemble = VotingClassifier(estimators=algoritma, voting='hard')

# Lakukan tuning hyperparameter untuk setiap algoritma
param_grid = {
    'Logistic Regression__C': [0.1, 1, 10],
    'SVM kernel polynomial__C': [0.1, 1, 10],
    'Decision Tree__max_depth': [None, 5, 10]
}

In [9]:
grid_search = GridSearchCV(ensemble, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Cetak hasil tuning hyperparameter
print("Hasil tuning hyperparameter:")
print(grid_search.best_params_)
print("Akurasi terbaik:", grid_search.best_score_)

Hasil tuning hyperparameter:
{'Decision Tree__max_depth': 5, 'Logistic Regression__C': 0.1, 'SVM kernel polynomial__C': 1}
Akurasi terbaik: 0.773610555777689


In [10]:
# Lakukan prediksi dengan ensemble voting
y_pred = grid_search.best_estimator_.predict(X_test)

# Cetak hasil prediksi
print("Hasil prediksi:")
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan klasifikasi:")
print(classification_report(y_test, y_pred))
print("Matriks konfusi:")
print(confusion_matrix(y_test, y_pred))

Hasil prediksi:
Akurasi: 0.7597402597402597
Laporan klasifikasi:
              precision    recall  f1-score   support

           0       0.79      0.85      0.82        99
           1       0.69      0.60      0.64        55

    accuracy                           0.76       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.75      0.76      0.76       154

Matriks konfusi:
[[84 15]
 [22 33]]
